#Running simple example through EC2 using spot pricing
To work one need to have spot instances running

In [1]:
%%bash
. ~/.bashrc
pip install  --upgrade git+https://git@github.com/JonasWallin/linkingEC2

  Cloning https://git@github.com/JonasWallin/linkingEC2 to /var/folders/ct/zpjk2t696k5bv03sqq9xrpd00000gn/T/pip-ADk4_e-build
Requirement already up-to-date: boto in /Users/jonaswallin/anaconda/lib/python2.7/site-packages (from linkingEC2==0.1)
  Found existing installation: linkingEC2 0.1
    Uninstalling linkingEC2-0.1:
      Successfully uninstalled linkingEC2-0.1
  Running setup.py install for linkingEC2


In [2]:
from linkingEC2 import LinkingHandler
from ConfigParser import ConfigParser

config = ConfigParser()
starfigconfig_folder = "/Users/jonaswallin/.starcluster/"
config.read(starfigconfig_folder + "config")

acess_key_id     = config.get('aws info', 'aws_access_key_id'    , 0)
aws_secret_key   = config.get('aws info', 'aws_secret_access_key', 0)
aws_region_name  = config.get('aws info', 'aws_region_name'      , 0)
my_key_loc       = config.get('key mykeyABC', 'key_location',0)


linker = LinkingHandler(aws_secret_access_key = aws_secret_key,
                        aws_access_key_id     = acess_key_id,
                        aws_region_name       = aws_region_name,
                        key_location          = my_key_loc,
                        key_name              = 'mykeyABC' )

In [3]:
linker.connect_spot_instances()

checking if ssh into node005 works:
*
checking if ssh into node006 works:
*
checking if ssh into node007 works:
*
checking if ssh into node008 works:
*
checking if ssh into node009 works:
*
checking if ssh into node010 works:
*
collecting node005 number of processes:done
collecting node006 number of processes:done
collecting node007 number of processes:done
collecting node008 number of processes:done
collecting node009 number of processes:done
collecting node010 number of processes:done
copying ~/.ssh/mykeyABC.rsa to external ~/.ssh/id_rsa
copying files to node005 done
copying files to node006 done
copying files to node007 done
copying files to node008 done
copying files to node009 done
copying files to node010 done
disable StrictHostKeyChecking  in node005:appending to ~/nodefile in node005 done
appending to /etc/hosts in node005 done


In [4]:
PACKAGES_APT = [' libatlas3-base',
                'libatlas-base-dev',
                'python-dev', 
                'openmpi-bin', 
                'libopenmpi-dev', 
                'python-numpy',
                'python-sklearn',
               'python-matplotlib',
               'git',
                'python-scipy',
               'r-base',
               'r-base-core']
PACKAGES_PIP = ['cython',
                'mpi4py',
                'simplejson',
               'rpy2']

#Adding later version of R
#http://philipp-burckhardt.com/2014/05/25/installing-r-rstudio-on-ubuntu/
command = 'sudo add-apt-repository "deb http://cran.rstudio.com/bin/linux/ubuntu trusty/"'
linker.send_command_ssh(command = 'gpg --keyserver pgpkeys.mit.edu --recv-key 51716619E084DAB9')
linker.send_command_ssh(command = 'gpg -a --export 51716619E084DAB9 | sudo apt-key add -')
linker.send_command_ssh(command = command)



linker.apt_install(PACKAGES_APT)

#problem with memory installing scipy:
#http://naokiwatanabe.blogspot.se/2014/12/install-numpy-schipy-matplotlib-and-etc.html
linker.send_command_ssh(command = 'sudo /bin/dd if=/dev/zero of=/var/swap.1 bs=1M count=1024')
linker.send_command_ssh(command = 'sudo /sbin/mkswap /var/swap.1')
linker.send_command_ssh(command = 'sudo /sbin/swapon /var/swap.1')
linker.pip_install('-U scipy')

linker.send_command_ssh(command = 'sudo swapoff /var/swap.1')
linker.send_command_ssh(command = 'sudo sudo rm /var/swap.1')



linker.pip_install(PACKAGES_PIP)
linker.pip_install(['git+https://git@github.com/JonasWallin/BayesFlow'])
import os
os.system('say "your packages is downloaded"')

node005, sshing in  gpg --keyserver pgpkeys.mit.edu --recv-key 51716619E084DAB9 
node005 done
node006, sshing in  gpg --keyserver pgpkeys.mit.edu --recv-key 51716619E084DAB9 
node006 done
node007, sshing in  gpg --keyserver pgpkeys.mit.edu --recv-key 51716619E084DAB9 
node007 done
node008, sshing in  gpg --keyserver pgpkeys.mit.edu --recv-key 51716619E084DAB9 
node008 done
node009, sshing in  gpg --keyserver pgpkeys.mit.edu --recv-key 51716619E084DAB9 
node009 done
node010, sshing in  gpg --keyserver pgpkeys.mit.edu --recv-key 51716619E084DAB9 
node010 done
node005, sshing in  gpg -a --export 51716619E084DAB9 | sudo apt-key add - 
node005 done
node006, sshing in  gpg -a --export 51716619E084DAB9 | sudo apt-key add - 
node006 done
node007, sshing in  gpg -a --export 51716619E084DAB9 | sudo apt-key add - 
node007 done
node008, sshing in  gpg -a --export 51716619E084DAB9 | sudo apt-key add - 
node008 done
node009, sshing in  gpg -a --export 51716619E084DAB9 | sudo apt-key add - 
node009 d

0

In [ ]:
linker.send_command_ssh( command = 'wget https://raw.githubusercontent.com/JonasWallin/BayesFlow/master/examples/article1/covs_.npy')
linker.send_command_ssh( command = 'wget https://raw.githubusercontent.com/JonasWallin/BayesFlow/master/examples/article1/means_.npy')
linker.send_command_ssh( command = 'wget https://raw.githubusercontent.com/JonasWallin/BayesFlow/master/examples/article1/weights_.npy')
linker.send_command_ssh( command = 'wget https://raw.githubusercontent.com/JonasWallin/BayesFlow/master/examples/article1/article_util.py')
linker.send_command_ssh( command = 'wget https://raw.githubusercontent.com/JonasWallin/BayesFlow/master/examples/article1/article_simulatedata.py')
linker.send_command_ssh( command = 'wget https://raw.githubusercontent.com/JonasWallin/BayesFlow/master/examples/article1/article_estimate_largerdata1_mpi.py')

node005, sshing in  wget https://raw.githubusercontent.com/JonasWallin/BayesFlow/master/examples/article1/covs_.npy 
node005 done
node006, sshing in  wget https://raw.githubusercontent.com/JonasWallin/BayesFlow/master/examples/article1/covs_.npy 
node006 done
node007, sshing in  wget https://raw.githubusercontent.com/JonasWallin/BayesFlow/master/examples/article1/covs_.npy 
node007 done
node008, sshing in  wget https://raw.githubusercontent.com/JonasWallin/BayesFlow/master/examples/article1/covs_.npy 
node008 done
node009, sshing in  wget https://raw.githubusercontent.com/JonasWallin/BayesFlow/master/examples/article1/covs_.npy 
node009 done
node010, sshing in  wget https://raw.githubusercontent.com/JonasWallin/BayesFlow/master/examples/article1/covs_.npy 
node010 done
node005, sshing in  wget https://raw.githubusercontent.com/JonasWallin/BayesFlow/master/examples/article1/means_.npy 
node005 done
node006, sshing in  wget https://raw.githubusercontent.com/JonasWallin/BayesFlow/master/e

0

In [ ]:
import numpy as np
tot_process = np.sum([node['n_process'] for node in linker.nodes])
command = 'mpirun -hostfile nodefile -n %d python article_estimate_largerdata1_mpi.py'%(tot_process)
linker.send_command_ssh(nodes = 0, command = command)
linker.copy_files_from_node('simulation_result.npy')
linker.copy_files_from_node('mus_sim.npy')
linker.copy_files_from_node('sim_data.npy')

node005, sshing in  mpirun -hostfile nodefile -n 192 python article_estimate_largerdata1_mpi.py 


[{'name': 'node003', 'private_dns': '', 'private_ip': None, 'public_dns': ''},
 {'name': 'node004', 'private_dns': '', 'private_ip': None, 'public_dns': ''},
 {'name': 'node005',
  'private_dns': u'ip-172-31-3-206.ec2.internal',
  'private_ip': u'172.31.3.206',
  'public_dns': u'ec2-52-23-172-34.compute-1.amazonaws.com'},
 {'name': 'node006',
  'private_dns': u'ip-172-31-11-42.ec2.internal',
  'private_ip': u'172.31.11.42',
  'public_dns': u'ec2-54-173-119-178.compute-1.amazonaws.com'},
 {'name': 'node007',
  'private_dns': u'ip-172-31-8-18.ec2.internal',
  'private_ip': u'172.31.8.18',
  'public_dns': u'ec2-54-173-14-35.compute-1.amazonaws.com'},
 {'name': 'node008',
  'private_dns': u'ip-172-31-10-26.ec2.internal',
  'private_ip': u'172.31.10.26',
  'public_dns': u'ec2-54-173-68-139.compute-1.amazonaws.com'},
 {'name': 'node009',
  'private_dns': u'ip-172-31-6-39.ec2.internal',
  'private_ip': u'172.31.6.39',
  'public_dns': u'ec2-54-173-95-221.compute-1.amazonaws.com'},
 {'name': 'n